In [2]:
#%%writefile get_proxy.py
#%load get_bookinfo.py
#%run get_bookinfo.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor 

from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq

#
import requests
import pandas as pd
import numpy as np
import random
import re
import json
import csv

#https://free-proxy-list.net/
def get_proxy(which='free',now=False,sample=False,sampleN=0):
    which_dict={'kuai':"/home/pan/django_projects/wtb/ips_kuai.txt",
                'ihuan':"/home/pan/django_projects/wtb/ips_1000_ihuan.txt",
                'free':"/home/pan/django_projects/wtb/ips_free.txt",
                'OK':"/home/pan/django_projects/wtb/ips_OKs.txt"
               }
    ippos=[]
    with open(which_dict[which], 'r') as f:
        lines=f.readlines()
        for line in lines:
            line=line.strip() #remove space_\t_\n
            if len(line.split(":"))==2:
                ippos.append(line)
    #
    if sample and sampleN>0:        
        return random.sample(ippos,sampleN)
    if now:
        s=0.5+random.uniform(0, 1)
        sleep(s)
        return random.choice(ippos)
    #ippo
    old=[]
    #測試代理_________________________________________
    while True:
        fake_header = Headers(
            browser="chrome",  # Generate only Chrome UA
            os="win",  # Generate ony Windows platform
            headers=True  # generate misc headers
        )    
        UA=fake_header.generate()
        #UA = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        #已經測過的就不要
        ippo=random.choice(ippos)
        if len(old)/len(ippos) > 0.5:
            old=[]
        if ippo in old:
            continue
        else:
            old.append(ippo)
        #    
        ip=ippo.split(":")[0]
        proxies={
                "http": "http://"+ippo,
                "https": "http://"+ippo
                }  
        #print("test:"+ippo)    
        #clear_output(wait=True)
        try:
            test=['http://icanhazip.com/','https://myip.com.tw/','https://www.showmyipaddress.eu/']
            r = requests.get(random.choice(test),#test[1], 
                             headers=UA,
                             proxies=proxies,
                             timeout=4)
            r.encoding='utf8'
            #print(r.status_code)
            #print(r.text)
            #return
            #return 22
            #
            if(r.status_code == 200 and (ip in r.text or '&#46;'.join(ip.split('.')) in r.text) ):
                print("OK:"+which+"_"+ippo)
                #print(r.status_code)
                #print(r.text)
                r.close() 
                if which!='OK':
                    fn='/home/pan/django_projects/wtb/ips_OKs.txt'
                    mode=0                    
                    if mode==0:
                        #加到最後一行
                        with open(fn, 'a+') as f:
                            f.write(ippo+"\r\n")      
                    else:
                        #刪掉第一行_加到最後一行
                        with open(fn, 'r+') as f: # open file in read / write mode
                            firstLine = f.readline() # read the first line and throw it out
                            data = f.read() # read the rest
                            data += ippo+"\r\n"
                            f.seek(0) # set the cursor to the top of the file
                            f.write(data) # write the data back
                            f.truncate() # set the file size to the current size 
                #______________
                return ippo
            else:#if not (r.status_code == 200 and ip in r.text):
                r.close()
                raise Exception('fail')
               
        except Exception as e:
            #print(str(e))
            #clear_output(wait=True) 
             
            sleep(1.5+random.uniform(0, 2))
            #sleep(0.1)
            continue
        #else:
        #    print(r.text)
        #    print("OK2:"+ip)
        #    r.close()       
        #    break    

In [3]:
if __name__ == '__main__':
    which_dict={
                #'kuai':"ips_kuai.txt",
                #'ihuan':"ips_1000_ihuan.txt",
                'free':'ips_free.txt',
                #'OK':'ips_OKs.txt'
               }
    for i in range(100):
        print(i)
        with ThreadPoolExecutor(max_workers=3) as executor:
            executor.map(get_proxy,which_dict.keys())

0
OK:free_189.57.62.146:80
1
OK:free_115.124.78.142:38994
2
OK:free_103.231.218.30:47568
3
OK:free_118.173.232.215:43186
4
OK:free_182.16.171.1:53281
5
OK:free_176.62.188.158:56351
6
OK:free_182.52.74.76:34084
7
OK:free_189.57.62.146:80
8
OK:free_181.117.176.236:61358
9
OK:free_103.22.248.59:61661
10
OK:free_154.127.120.18:30280
11
OK:free_119.81.71.27:8123
12
OK:free_186.0.176.147:8080
13
OK:free_51.38.71.101:8080
14
OK:free_118.173.233.151:40025
15
OK:free_118.174.232.237:48665
16
OK:free_177.12.80.50:50556
17
OK:free_178.216.0.168:33364
18
OK:free_81.163.62.136:41258
19
OK:free_80.187.140.26:8080
20
OK:free_103.239.254.70:61967
21
OK:free_159.8.114.37:8123
22
OK:free_36.37.113.4:36485
23
OK:free_80.187.140.26:8080
24
OK:free_168.90.50.118:39734
25
OK:free_200.206.50.66:42515
26
OK:free_5.59.141.152:61981
27
OK:free_103.225.206.22:31777
28
OK:free_85.31.40.46:8080
29
OK:free_176.117.255.182:53100
30
OK:free_186.10.65.3:999
31
OK:free_118.175.207.25:38325
32
OK:free_168.90.50.118:3973

In [ ]:
#kuai,ihuan,free
%time get_proxy('OK',True,sample=True,sampleN=3)

In [ ]:
with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'r') as f:
    lines=f.readlines()
    for l in lines:
        print(l.strip()+"_")
        #print(l+"__")

In [ ]:
ippos=[]
with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip() #remove space_\t_\n
        if len(line.split(":"))==2:
            ippos.append(line)